In [1]:
import pyspark as ps
import json
import hashlib

from pyspark.sql import SQLContext
from pyspark.mllib.recommendation import ALS

import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    sc = ps.SparkContext('local[*]')
except:
    warnings.warn("SparkContext already exists in this scope")

/home/lplab/anaconda3/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


# Load Dataset

In [3]:
fields = ['product_id',
           'user_id',
           'review',
           'profile_name',
           'helpfulness',
           'score',
           'time']

In [4]:
def validate(line):
    for field in fields:
        if field not in line :
            return False
    return True

In [5]:
reviews_raw = sc.textFile('./movies 1.json')
reviews = reviews_raw.map(lambda line: json.loads(line)).filter(validate)
reviews.cache()

PythonRDD[2] at RDD at PythonRDD.scala:53

In [6]:
reviews.take(1)

[{'user_id': 'A141HP4LYPWMSR',
  'product_id': 'B003AI2VGA',
  'review': 'Synopsis: On the daily trek from Juarez, Mexico to El Paso, Texas an ever increasing number of female workers are found raped and murdered in the surrounding desert. Investigative reporter Karina Danes (Minnie Driver) arrives from Los Angeles to pursue the story and angers both the local police and the factory owners who employee the undocumented aliens with her pointed questions and relentless quest for the truth.<br /><br />Her story goes nationwide when a young girl named Mariela (Ana Claudia Talancon) survives a vicious attack and walks out of the desert crediting the Blessed Virgin for her rescue. Her story is further enhanced when the "Wounds of Christ" (stigmata) appear in her palms. She also claims to have received a message of hope for the Virgin Mary and soon a fanatical movement forms around her to fight against the evil that holds such a stranglehold on the area.<br /><br />Critique: Possessing a life

In [7]:
def get_hash(s):
    return int(hashlib.sha1(s).hexdigest(), 16) % (10 ** 8)

In [8]:
ratings = reviews.map(lambda entry: tuple([get_hash(entry['user_id'].encode('utf-8')), get_hash(entry['product_id'].encode('utf-8')), int(entry['score'])]))

train_data = ratings.filter(lambda entry: ((entry[0]+entry[1]) % 10) >= 2)
test_data = ratings.filter(lambda entry: ((entry[0]+entry[1]) % 10) < 2)
train_data.cache()

PythonRDD[4] at RDD at PythonRDD.scala:53

In [9]:
print(f'Num of train samples: {train_data.count()}')
print(f'Num of test samples: {test_data.count()}')

Num of train samples: 39992
Num of test samples: 10008


# Train ALS model

In [12]:
rank = 20
numIterations = 20
model = ALS.train(train_data, rank, numIterations)

In [13]:
def convertToFloat(lines):
    returnedLine = []
    for x in lines:
        returnedLine.append(float(x))
    return returnedLine

In [14]:
unknown = test_data.map(lambda entry: (int(entry[0]), int(entry[1])))
predictions = model.predictAll(unknown).map(lambda r: ((int(r[0]), int(r[1])), r[2]))
true_and_predictions = test_data.map(lambda r: ((int(r[0]), int(r[1])), r[2])).join(predictions)

MSE = true_and_predictions.map(lambda r: (int(r[1][0]) - int(r[1][1]))**2).reduce(lambda x,y: x+y)/true_and_predictions.count()

In [15]:
true_and_predictions.take(5)

[((65965270, 62577830), (5, -1.3322661454208051)),
 ((7383110, 62577830), (5, 1.0943978617188526)),
 ((22477285, 58302865), (4, 0.965967031753221)),
 ((5479805, 58302865), (5, 0.4529040479812738)),
 ((39998009, 30926631), (3, 2.9505422867277344))]

In [16]:
print(MSE)

8.901917199222007
